# Simulado para AF de MEGADADOS

Utilize este notebook como uma das etapas de preparação para a Avaliação Final. Considere que foi realizado um recorte dos conteúdos e assuntos não cobertos neste notebook ainda poderão ser cobrados.

**NOME**: Antônio Amaral Egydio Martins

## Insper autograding!

Para receber feedback dos exercício na parte 1, iremos utilizar o `insper autograding`. Se não fez este passo, abra o notebook disponível na aula 09.

**Sugestão**: para aproveitar o `.env`, crie uma pasta para a prova no mesmo local onde costuma deixar as pastas das **aulas**!

## Como resolver os exercícios?

Crie a base da prova em sua máquina (Exercício 1). Utilize o notebook, MySQL Workbench ou o conector para testar as queries e soluções. Quando estiver bastante certo de que a resposta está correta, faça a submissão para o servidor.

## Import das bibliotecas

Vamos realizar o import das bibliotecas.

In [6]:
import mysql.connector
from functools import partial
import os
import insperautograder.jupyter as ia
from dotenv import load_dotenv
from functools import reduce

E vamos criar nosso HELPER de conexão com o banco! Perceba que, uma vez configurado o `.env` não precisaremos mais informar usuários, senhas e URLs!

In [7]:
load_dotenv(override=True)

def get_connection_helper():

    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            print('Executando query:')
            cursor.execute(query, args)
            for result in cursor:
                print(result)

    connection = mysql.connector.connect(
        host=os.getenv('MD_DB_SERVER'),
        user=os.getenv('MD_DB_USERNAME'),
        password=os.getenv('MD_DB_PASSWORD'),
        database='parques',
    )
    return connection, partial(run_db_query, connection)


connection, db = get_connection_helper()

### Notas

Para conferir a nota da correção automática da prova, utilize:

In [8]:
ia.grades(task='revisao_af_md_1')

|    | Atividade       | Exercício   |   Peso |   Nota |
|---:|:----------------|:------------|-------:|-------:|
|  0 | revisao_af_md_1 | ex01a       |      1 |     10 |
|  1 | revisao_af_md_1 | ex01b       |      1 |     10 |
|  2 | revisao_af_md_1 | ex01c       |      1 |     10 |
|  3 | revisao_af_md_1 | ex01d       |      1 |      0 |
|  4 | revisao_af_md_1 | ex01f       |      3 |      0 |

In [41]:
ia.grades(by='TASK')

|    | Tarefa               |   Nota |
|---:|:---------------------|-------:|
|  0 | agg_join             |  10    |
|  1 | ai_md_23_2           |  10    |
|  2 | ddl                  |   3.33 |
|  3 | desafio_normalizacao |  10    |
|  4 | dml                  |  10    |
|  5 | exercicios_spark     |  10    |
|  6 | functional           |  10    |
|  7 | group_having         |  10    |
|  8 | newborn              |  10    |
|  9 | permissions          |  10    |
| 10 | revisao_af_md_1      |  10    |
| 11 | select01             |  10    |
| 12 | spark                |  10    |
| 13 | sql_review1          |  10    |
| 14 | triggers             |  10    |
| 15 | views                |  10    |

In [10]:
ia.tasks()

|    | Atividade            | De                        | Até                       |
|---:|:---------------------|:--------------------------|:--------------------------|
|  0 | select01             | 2023-08-08 03:00:00+00:00 | 2023-08-21 02:59:59+00:00 |
|  1 | newborn              | 2023-08-08 03:00:00+00:00 | 2023-08-16 02:59:59+00:00 |
|  2 | ddl                  | 2023-08-27 03:00:00+00:00 | 2023-09-02 02:59:59+00:00 |
|  3 | dml                  | 2023-08-29 03:00:00+00:00 | 2023-09-04 02:59:59+00:00 |
|  4 | group_having         | 2023-09-03 03:00:00+00:00 | 2023-09-17 02:59:59+00:00 |
|  5 | agg_join             | 2023-09-03 03:00:00+00:00 | 2023-09-09 02:59:59+00:00 |
|  6 | views                | 2023-09-11 03:00:00+00:00 | 2023-09-18 02:59:59+00:00 |
|  7 | sql_review1          | 2023-09-13 03:00:00+00:00 | 2023-09-20 02:59:59+00:00 |
|  8 | permissions          | 2023-09-20 03:00:00+00:00 | 2023-09-27 03:00:00+00:00 |
|  9 | desafio_normalizacao | 2023-09-25 03:00:00+00:00 | 2023-10-02 03:00:00+00:00 |
| 10 | ai_md_23_2           | 2023-10-09 03:00:00+00:00 | 2023-10-10 03:00:00+00:00 |
| 11 | triggers             | 2023-10-19 03:00:00+00:00 | 2023-10-28 03:00:00+00:00 |
| 12 | spark                | 2023-10-29 03:00:00+00:00 | 2023-11-08 15:00:00+00:00 |
| 13 | functional           | 2023-10-29 03:00:00+00:00 | 2023-11-08 15:00:00+00:00 |
| 14 | exercicios_spark     | 2023-11-02 03:00:00+00:00 | 2023-11-14 03:00:00+00:00 |
| 15 | revisao_af_md_1      | 2023-11-27 03:00:00+00:00 | 2023-12-05 03:00:00+00:00 |

**Exercício 1**:

Em uma cidade temos vários parques públicos. A prefeitura resolveu privatizar a zeladoria de todos eles: existe um conjunto de empresas administradoras, cada uma delas responsável por alguns parques da cidade. Para monitorar o trabalho dessas empresas, a prefeitura criou o aplicativo "**Fiscais do Parque**", onde usuários podem notificar ocorrências indesejáveis.

O modelo entidade-relacionamento que representa esse cenário é visto abaixo:

<img src="img/modelo_er.png">

O modelo relacional correspondente ao *design* acima é visto a seguir:

<img src="img/modelo_relacional.png">

O script `pf.sql` contém o código para criar o banco de dados `parques` no MySQL e também adiciona dados de exemplo, usem a vontade para testar as respostas das questões da prova!

**a)** Escreva uma query que liste todas as administradoras (apenas o `nome`), ordenadas por número de eventos de segurança ocorridos nos parques que administram. Pegadinha: e se não houve evento de segurança em seus parques?

In [11]:
sql_ex01a = '''
SELECT
    administradora.nome
FROM
    administradora
    INNER JOIN parque USING(id_adm)
    LEFT JOIN evento USING(id_parque)
GROUP BY
    administradora.id_adm
ORDER BY
    COUNT(evento.id_evento) ASC;
'''

db(sql_ex01a)

Executando query:
('Associação Autônoma Andarilha',)
('Instituto Instigare',)
('Sociedade dos Sócios',)
('Grupo Gripe',)
('Fundação Fundão',)


In [12]:
ia.sender(answer='sql_ex01a', task='revisao_af_md_1', question='ex01a', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex01a', style=ButtonStyle()), Output()), _dom_classes=('widge…

**b)** Crie uma tabela temporária chamada `tmp_total` que tem as colunas `(id_usuario, total)` onde a coluna `total` indica o numero total de eventos notificados pelo usuario `id_usuario`.

In [13]:
sql_ex01b = '''
CREATE TEMPORARY TABLE tmp_total
    SELECT
        usuario.id_usuario, COUNT(evento.id_evento) as total
    FROM
        usuario
        INNER JOIN evento USING (id_usuario)
    GROUP BY
        usuario.id_usuario
'''

db(sql_ex01b)

Executando query:


In [14]:
ia.sender(answer='sql_ex01b', task='revisao_af_md_1', question='ex01b', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex01b', style=ButtonStyle()), Output()), _dom_classes=('widge…

**c)** Crie uma tabela temporária chamada `tmp_usuario_parque` que tem as colunas (`id_usuario`, `id_parque`, `contagem`) onde `contagem` é o número de eventos que o usuário `id_usuario` notificou acerca do parque `id_parque`.

Dica: `GROUP BY` permite agrupar por multiplas colunas.

**Obs**:
- Ordene de forma crescente por `id_usuario`, `id_parque`

In [15]:
sql_ex01c = '''
CREATE TEMPORARY TABLE tmp_usuario_parque
    SELECT
        usuario.id_usuario, parque.id_parque, COUNT(evento.id_evento) as contagem
    FROM
        usuario
        INNER JOIN evento USING (id_usuario)
        INNER JOIN parque USING (id_parque)
    GROUP BY
        usuario.id_usuario, parque.id_parque
    ORDER BY
        usuario.id_usuario ASC, parque.id_parque ASC;
'''

# sql_ex01c = '''

# '''

db(sql_ex01c)

Executando query:


Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [16]:
ia.sender(answer='sql_ex01c', task='revisao_af_md_1', question='ex01c', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex01c', style=ButtonStyle()), Output()), _dom_classes=('widge…

**d)** Um usuário é um **fã** de um parque se a fração de eventos que ele notifica para o parque, em relação ao número total de eventos que ele notificou para todos os parques, é superior a 20%. Usando as tabelas temporárias acima (mesmo que não as tenha desenvolvido) faça uma query que lista os usuários que são fãs de dois ou mais parques.

**Obs**:

- Suponha que as tabelas temporárias já existem, faça apenas a query que retorne o `id_usuario` dos fãs!
- Ordene de forma crescente pelo `id_usuario`

In [39]:
sql_ex01d = '''
SELECT
    tmp_total.id_usuario
FROM
    tmp_total
INNER JOIN
    tmp_usuario_parque USING(id_usuario)
WHERE
    (tmp_usuario_parque.contagem / tmp_total.total) >= 0.2
GROUP BY
    tmp_total.id_usuario
HAVING
    COUNT(DISTINCT tmp_usuario_parque.id_parque) >= 2
ORDER BY
    tmp_total.id_usuario ASC;
'''

db(sql_ex01d)

Executando query:
(26,)
(36,)


Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [ ]:
ia.sender(answer='sql_ex01d', task='revisao_af_md_1', question='ex01d', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex01d', style=ButtonStyle()), Output()), _dom_classes=('widge…

**e)** Nos itens (b), (c) e (d) construímos uma solução para encontrar os fãs dos parques através do uso de duas tabelas temporárias e uma query. Essa solução tem uma falha: pode ser que entre a criação da tabela `tmp_total` e da tabela `tmp_usuario_parque` tivemos a chegada de novos eventos!

Explique porque isso causa uma inconsistência na solução, e proponha um jeito de resolver esse problema (não precisa implementar, apenas propor a forma de resolver – mas seja preciso na especificação!).

Esta questão será corrigida considerando a seguinte subrica:

| Conceito | Nota | Descrição                                                                                                                                               |
|:----------:|----------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------|
| I        | 0.0 |Apenas citou o assunto ou alguns fatos sem explicações                                                                                                  |
| D        | 0.3 |Explicou superficialmente o assunto ou fatos mas sem muitos detalhes conclusivos                                                                        |
| C        | 0.6 |Explicou com detalhes, apresentando definições concretas.                                                                                               |
| B        | 0.8 |Explicou com detalhes, apresentando definições concretas e exemplos de uso.                                                                            |
| A        | 1.0 |Explicou com detalhes, apresentando definições concretas, exemplos de uso e ainda outros tópicos correlatos, fazendo uma conexão lógica entre eles. |

<div class="alert alert-success">

Uma vez que estamos utilizando duas tabelas temporarias para a criação de um Query, neste caso para saber a fidelidade de um cliente, podem haver múltiplos pontos de inconsistência, sendo o primeiro deles a má formação das tabelas temporarias, porém outros problemas mais profundos podem surgir como por exemplo em um sistema com múltiplos usuários.

Digamos que no momento que o Administrador foi utilizar ambas as tabelas para ter seu resultado houve alteração em números do sistema, porém como as tabelas temporarias já estavam criadas, o resultado não seria o esperado, visto que a contagem estaria desatualizada e não refletiria a realidade.

</div>

**f)** Algum engenheiro da sua equipe propôs adicionar uma coluna chamada `cont_eventos` à tabela `aministradora` para manter a contagem total de eventos em parques sob administração de cada empresa.

Obviamente trata-se de uma redundância no nosso banco de dados, e redundâncias são perigosas: podem resultar em uma inconsistência. Mas, por algum motivo bizarro (performance talvez?), você não tem alternativa a não ser implementar essa *feature*.

Crie uma **trigger** que incremente a coluna `cont_eventos` na `administradora` adequada sempre que um novo `evento` for inserido.

**Obs**:
- Idealmente, a coluna precisaria ser decrementada quando houvesse uma remoção de evento. Entretanto, nesta questão iremos considerar apenas o caso de inserção!
- Crie a coluna `cont_eventos` em seu database local para testar
- No servidor, considere que a coluna `cont_eventos` já existe. Submeta apenas o código de criação da trigger.
- Faça vários inserts para testar!

In [28]:
sql_ex01f = '''
CREATE TRIGGER atualiza_contagem_eventos
    AFTER INSERT ON evento
    FOR EACH ROW
    BEGIN

        UPDATE administradora
            SET cont_eventos = cont_eventos + 1
        WHERE
            id_adm = (SELECT id_adm FROM parque WHERE id_parque = NEW.id_parque);

    END;
'''


db(sql_ex01f)

Executando query:


DatabaseError: 1359 (HY000): Trigger already exists

Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [ ]:
ia.sender(answer='sql_ex01f', task='revisao_af_md_1', question='ex01f', answer_type='pyvar')

interactive(children=(Button(description='Enviar ex01f', style=ButtonStyle()), Output()), _dom_classes=('widge…

**Exercício 2**:

Explique com suas palavras: porque o Spark é baseado em um paradigma de programação funcional?

Esta questão será corrigida considerando a seguinte subrica:

| Conceito | Nota | Descrição                                                                                                                                               |
|:----------:|----------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------|
| I        | 0.0 |Apenas citou o assunto ou alguns fatos sem explicações                                                                                                  |
| D        | 0.3 |Explicou superficialmente o assunto ou fatos mas sem muitos detalhes conclusivos                                                                        |
| C        | 0.6 |Explicou com detalhes, apresentando definições concretas.                                                                                               |
| B        | 0.8 |Explicou com detalhes, apresentando definições concretas e exemplos de uso.                                                                            |
| A        | 1.0 |Explicou com detalhes, apresentando definições concretas, exemplos de uso e ainda outros tópicos correlatos, fazendo uma conexão lógica entre eles. |

<div class="alert alert-success">

Sua resposta AQUI!

</div>

**Exercício 3**:

Escreva uma função `func_ex` em Python usando apenas ferramentas de programação funcional (como map(), functools.reduce(), filter(), zip(), list comprehensions, etc.) para a seguinte tarefa: 

- São dadas duas listas $x$ e $y$ de números reais, de mesmo comprimento . Cada elemento de cada lista é um número real ou `None`, indicando que o número está ausente.

- A função deverá calcular a soma a seguir:

$$\sum_{
\begin{matrix}
i=1,2,\dots,n\\ 
x_i \neq \text{None}\\ 
y_i \neq \text{None}
\end{matrix}}{(x_i - y_i)^2}$$

Ou seja, a soma dos quadrados das diferenças entre os valores de $x$ e $y$, mas apenas quando ambos os valores não são `None`.

**Obs**:
- Na matemática o primeiro índice é 1. Considere que todos os pares de items devem ser submetidos ao cálculo
- Utilize funções aninhadas ou internas se necessário
- Não pode utilizar `for`. Tente fazer sem `sum` (crie versão com `reduce`)!

In [99]:
# Altere esta função
def func_ex(x, y):
    x = map(lambda x: x if x != None else 0, x)
    y = map(lambda y: y if y != None else 0, y)
    vec = map(lambda x: ((x[0] if x[1] != 0 else 0) - (x[1] if x[0] != 0 else 0))**2, zip(x, y))
    res = reduce(lambda x, y: x + y, vec)
    print(res)
    return res

def func_ex(x, y):
    return reduce(
        lambda x, y: x + y,
        map(
            lambda x: (x[0] - x[1]) ** 2,
            filter(
                lambda x: x[0] != None and x[1] != None,
                zip(x, y),
            ),
        ),
        0,
    )

In [100]:
x = [1, 2, 3]
y = [1, 4, 6]
z = [+2, -2, -2, 0, None, None,   20]
w = [-2, -2, +2, 0,    5, None, None]

# Alguns testes, objetivo é não dar Exception!
assert func_ex(x,x) == 0  # de x para x deve ser zero
assert func_ex(y,y) == 0  # de y para y deve ser zero
assert func_ex(x,y) == 13 # de x para y deve ser 13
assert func_ex(y,x) == func_ex(x,y) # de x para y deve ser igual dist. y para x

assert func_ex(z,z) == 0
assert func_ex(w,w) == 0
assert func_ex(z,w) == func_ex(w,z)
assert func_ex(z,w) == 32

**Exercício 4**:

O site https://lichess.org permite que pessoas do mundo todo joguem xadrez online (mesmo com deficiência visual!).

É gratuito, funciona com doações (lichess.org é uma instituição sem fins lucrativos na França) que servem para sustentar o custo dos servidores (web app, varios bancos de dados, sistemas de análise de jogos – e.g. stockfish – etc.) e o salário do desenvolvedor principal e criador da plataforma. O código-fonte é aberto, e desenvolvido principalmente em Scala (pelas vantagens da programação funcional, vejam só!).

Na página de doações (https://lichess.org/patron) tem uma planilha de custos que lista todos os servidores, suas capacidades e custos. Tem também um dado interessante: são 5.200.000 jogos por dia atualmente!

Suponha que cada jogo tem em média 50 lances, que cada lance pode ser armazenado como um par de bytes indicando a casa de origem e destino de uma peça. Suponha ainda que o número de jogos por ano dobra a cada ano. A planilha de custos do lichess informa que o servidor de banco de dados tem 3840 GB de disco; suponha que 1TB ainda esteja livre para armazenamento de dados. A partir de agora, quantos anos de armazenamento de jogos o servidor ainda aguenta?

<div class="alert alert-success">

1 Lance = 2 bytes
1 jogo = 50 lances -> 100 bytes
1 dia = 5.200.000 jogos * 100 bytes = 52.000.000 bytes ou 52 MB
1 ano = 365 * 52 MB = 18980 MB ou 18,980 GB

Como o caso é uma PG

Sn = a * ((r ** n) -1) / r-1

a = 52 MB
r = 365



</div>

**Exercício 5**:

Considerando a tabela de gastos para departamentos de filiais de uma empresa:

| id_filial (PK) | id_departamento (PK) | descricao  | id_cidade  | cidade | valor |
|----------------|----------------------|------------|------------|--------|-------|
| 10             | 55                   | TI         | 1          | Santos | 45.00 |
| 20             | 55                   | TI         | 2          | Jundiaí|  7.50 |
| 20             | 56                   | Marketing  | 2          | Jundiaí| 22.91 |
| 10             | 56                   | Marketing  | 1          | Santos |  6.12 |

**a)** A tabela está na `1NF`? Está na `2NF`? E na `3NF`? Justifique.

<div class="alert alert-success">

1NF, visto que nenhum dos campos têm tuplas com múltiplos valorea tribuitos, todos seguem a padronização de valores simples. Porém não está na normalização 2NF, visto que os atributos: descricao, id_cidade, cidade e valor não dependentem funcionamente da chave primaria (composta por id_filial e id_departamento). É possível ver esta não dependencia quando é comparada a coluna de valores que é variável, ou a coluna de descrição que independente da id_filial, ou id_departamento pode ser TI.

</div>

**b)** Caso não esteja na 3NF, proponha modificações até que esteja na 3NF.

<div class="alert alert-success">

Sua resposta AQUI!
    
Exemplo de tabela, crie quantas precisar!

| id_filial (PK) | id_departamento (PK) | descricao  | id_cidade  | cidade | valor |
|----------------|----------------------|------------|------------|--------|-------|
| 10             | 55                   | TI         | 1          | Santos | 45.00 |
| 20             | 55                   | TI         | 2          | Jundiaí| 45.00 |
| 20             | 56                   | Marketing  | 2          | Jundiaí| 22.91 |
| 10             | 56                   | Marketing  | 1          | Santos | 22.91 |

Você pode editar tabelas markdown em https://www.tablesgenerator.com/markdown_tables
</div>

### Conferindo as notas!

In [ ]:
ia.grades(task='revisao_af_md_1')

## Entrega!

Não precisa entregar na revisão, apenas na prova! Deixamos para fins didáticos!

É hora de entregar. Faça um **zip** do notebook + imagens chamado `revisao_af_megadados.zip` e submeta no Blackboard!

**Obs**:
- `zip`, não use `rar`!
- Não precisa entregar os scripts fornecidos pelo professor (pasta scripts)

## Gabarito

**<div id="gab_ex3">Exercício 3</div>**
<div class="alert alert-warning">
Solução sem funcional (não seria aceita como resposta):

```python
def func_ex(x, y):
    N = len(x)
    assert N == len(y)
    soma = 0.0
    for k in range(N):
        if x[k] != None and y[k] != None:
            soma += (x[k] - y[k])**2
    return soma
```

Algumas opções que poderiam ser consideradas como corretas:

```python
def func_ex(x, y):
    
    def diff(a,b):
        return 0 if a is None or b is None else (a - b) ** 2
    
    pairs = zip(x,y)
    sq_diffs = map(lambda pair: diff(pair[0], pair[1]), pairs)
    soma = reduce(lambda x, y: x + y, sq_diffs)
    return soma
```
    
```python
def func_ex(x, y):
    return reduce(
        lambda x, y: x + y,
        map(
            lambda x: (x[0] - x[1])**2,
            filter(
                lambda x: x[0] != None and x[1] != None,
                zip(x, y),
            ),
        ),
        0,
    )
```

Opção com `sum`:
```python
def func_ex(x, y):
    return sum(
        map(
            lambda x: (x[0] - x[1])**2,
            filter(
                lambda x: x[0] != None and x[1] != None,
                zip(x, y),
            ),
        ),
    )
```

</div>

**<div id="gab_ex4">Exercício 4</div>**

<div class="alert alert-warning">

O número de jogos por dia é dado por 
\[ f(t) = 5.2 \times 10^6 \times 2^{\frac{t}{365}}, \]
onde \( t \) é o número do dia. O número de bytes a serem armazenados por dia é
\[ g(t) = 50 \frac{\text{lances}}{\text{partida}} \times 2 \frac{\text{bytes}}{\text{lance}} \times f(t) \frac{\text{partidas}}{\text{dia}} = 5.2 \times 10^8 \times 2^{\frac{t}{365}}. \]

O volume total a ser armazenado no acúmulo de \( t \) dias é
\[ G(t) = \sum_{k = 0}^{t} g(t) = \sum_{k = 0}^{t} 5.2 \times 10^8 \times 2^{\frac{t}{365}} = \sum_{k = 0}^{t} 5.2 \times 10^8 \times \left(2^{\frac{1}{365}}\right)^t \]
que é a soma de uma progressão geométrica de termo inicial \( g(0) = 5.2 \times 10^8 \) e razão \( r = 2^{\frac{1}{365}} \).

Portanto, a fórmula fechada para \( G(t) \) é 
\[ G(t) = \frac{g(t + 1) - g(0)}{r - 1} = 5.2 \times 10^8 \frac{2^{\frac{t + 1}{365}} - 1}{2^{\frac{1}{365}} - 1} \]

Vamos ver o comportamento desta função:
    
```python
def G(t):
    return 5.2e8 * (2.0 ** ((t + 1.0) / 365.0) - 1.0) / (2.0 ** (1.0/365.0) - 1.0)

t = np.arange(1, 1000, 1)
y = G(t)
y_1TB = 1e12 * np.ones(t.shape)

plt.figure()
plt.plot(t, y, '-', color='blue', label='$G(t)$')
plt.plot(t, y_1TB, '--', color='orange', label='1 TB')

plt.title('Armazenamento utilizado')
plt.xlabel('dias')
plt.ylabel('bytes')
plt.legend()
plt.show()
